# Install

# Import

In [1]:
import pandas as pd
import re
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)
from modules.schedule import ITR1_SECTIONS

# fn

In [2]:
from modules.extract_logic import clean_row,extract_sections,apply_dynamic_headers,is_empty_row_specific

# Code

In [3]:
input_file_path = 'OUTPUT/ ITR 1-2 AY 2023-24.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
# excel_file_path = "output.xlsx"
config_path = "config/ITR1_header.json"
itr1 = ITR1_SECTIONS(input_file_path, output_file_path, config_path)
itr1

In [4]:
df_EXEMPT_INCOME= itr1.get_section("Acknowledgement_Number")
display(df_EXEMPT_INCOME)

""


In [5]:
# assert False

In [6]:
# itr1.export_to_excel(excel_file_path)

In [7]:
# itr1.start_pattern

In [8]:
# itr1.end_pattern

In [9]:
# itr1.extracted_data[33:34]

In [10]:
# df_temp = pd.DataFrame(itr1.extracted_data[33:34])
# df_temp = clean_row(df_temp)
# # df_temp = apply_dynamic_headers(df_temp, itr1.config,"ScheduleEA10_13A")
# df_temp

In [11]:
# itr1.start_pattern
# itr1.end_pattern
# itr1.hdr_row_map

In [12]:
# extract_sections(itr1.extracted_data[142:149], itr1.start_pattern,itr1.end_pattern,itr1.hdr_row_map)

In [13]:
itr1.sections

{'PART_E': {'start': 129, 'end': 132},
 'SCHEDULE_80G_A': {'start': 157, 'end': 159},
 'SCHEDULE_80G_B': {'start': 162, 'end': 165},
 'SCHEDULE_80G_C': {'start': 168, 'end': 173},
 'SCHEDULE_80G_D': {'start': 176, 'end': 178},
 'SCHEDULE_TDS1': {'start': 196, 'end': 199},
 'SCHEDULE_TDS2': {'start': 202, 'end': 204}}

In [14]:
# assert False

In [15]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

""


In [16]:
df_LTCG112A= itr1.get_section("LTCG112A")
display(df_LTCG112A)

""


In [17]:
# df_PART_C= itr1.get_section("PART_C")
# display(df_PART_C)

In [18]:
# df_PART_D= itr1.get_section("PART_D")
# display(df_PART_D)

In [19]:
df_PART_E= itr1.get_section("PART_E")
display(df_PART_E)

,RowNumber,IFSCodeoftheBank,NameoftheBank,AccountNumber,Typeofaccount
0,(1),(2),(3),(4),(5)
1,1,BARB0NAINIT,BANK OF BARODA,06090100019492,NaN


In [20]:
df_EA10_13A= itr1.get_section("ScheduleEA10_13A")
display(df_EA10_13A)

""


In [21]:
df_80u = itr1.get_section("SECTION_80U")
display(df_80u)

""


In [22]:
df_80dd = itr1.get_section("SECTION_80DD")
display(df_80dd)

""


In [23]:
df_80G_A= itr1.get_section("SCHEDULE_80G_A")
display(df_80G_A)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon100PercentOtherMode,TotDon100Percent,TotEligibleDon100Percent,(12)
0,(1),(2),(3),(4),(5),(6),(7),(9),(10),(11),(12)
1,Total A,None,None,None,None,None,None,0,0,0,0


In [24]:
df_80G_B= itr1.get_section("SCHEDULE_80G_B")
display(df_80G_B)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon50PercentNoApprReqdCash,TotDon50PercentNoApprReqdOtherMode,TotDon50PercentNoApprReqd,TotEligibleDon50Percent
0,(1),(2),(3),(4),(5),(6),(7),(9),(10),(11),(12)
2,Total B,None,None,None,None,None,None,0,0,0,0


In [25]:
df_80G_C= itr1.get_section("SCHEDULE_80G_C")
display(df_80G_C)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon100PercentApprReqdCash,None,TotDon100PercentApprReqdOtherMode,None,TotDon100PercentApprReqd,TotEligibleDon100PercentApprReqd
0,(1),(2),(3),(4),(5),(6),(7),(9),None,(10),None,(11),(12)
4,Total C,None,None,None,None,None,0,None,0,None,0,None,0


In [26]:
df_80G_D= itr1.get_section("SCHEDULE_80G_D")
display(df_80G_D)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,ARN(DonationReferenceNumber),TotDon100PercentApprReqdCash,TotDon100PercentApprReqdOtherMode,TotDon100PercentApprReqd,TotEligibleDon100PercentApprReqd
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12)
1,Total D,None,None,None,None,None,None,None,0,0,0,0


In [27]:
df_80GGA= itr1.get_section("SCHEDULE_80GGA")
display(df_80GGA)

""


In [28]:
df_80GGC= itr1.get_section("SCHEDULE_80GGC")
display(df_80GGC)

""


In [29]:
df_it= itr1.get_section("SCHEDULE_IT")
display(df_it)

""


In [30]:
df_TDS1= itr1.get_section("SCHEDULE_TDS1")
display(df_TDS1)

,RowNumber,TANofTheDeductor,NameofDeductor,IncomeChargeableSalaries,TaxPaid
0,(1),(2),(3),(4),(5)
1,1,MRTK01232A,KURMANCHAL NAGAR SAHKARI\nBANK LTD,"10,45,924","2,30,950"
2,Total,None,None,None,"2,30,950"


In [31]:
df_TDS2= itr1.get_section("SCHEDULE_TDS2")
display(df_TDS2)

,RowNumber,TANofTheDeductor,(3),GrossReceipt,YearofTaxDeduction,TaxDeducted,TDSCreditOutof(6)ClaimedThisYear
0,(1),(2),(3),(4),(5),(6),(7)
1,Total,None,None,None,None,None,0


In [32]:
df_TDS3= itr1.get_section("SCHEDULE_TDS3")
display(df_TDS3)

""


In [33]:
df_TCS= itr1.get_section("SCHEDULE_TCS")
display(df_TCS)

""
